In [ ]:
###################################################
### Publication or Commercial usage not allowed ###
###          v.rafanavicius@gmail.com           ###
###################################################

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import folium
import csv
import pandas as pd
import simplekml
import math
from noaa_sdk import noaa
import requests
from datetime import datetime, timedelta
from netCDF4 import num2date
from siphon.catalog import TDSCatalog
from metpy.units import units
import metpy.calc as mc

In [ ]:
#Sandbox for playing with pandas and kml files
data = {'Name': ['name1', 'name2'], 
        'Lat': [54.1, 55.1], 'Lon': [23.2,23.2],
        'Altitude': [100, 200],
        'Description':['description1', 'description2']}
df = pd.DataFrame(data=data)
kml = simplekml.Kml()
df.apply(lambda X: kml.newpoint(name=X["Name"], description=X["Description"], coords=[( X["Lon"],X["Lat"],X["Altitude"])]) ,axis=1)
kml.save(path = "data.kml")

In [ ]:
class Wind:
#     def __init(self, position, speed, direction, pressure):
    def __init__(self):
        self.position = np.array([23,54], dtype=np.float32)
        self.speed = 0
        self.direction = 0
        self.pressure = 100000
        self.names = ['u_wind, m/s', 'v_wind, m/s', 'pressure, Pa']
        
    # Copyright (c) 2013-2015 Siphon Contributors.
    # Distributed under the terms of the BSD 3-Clause License.
    # SPDX-License-Identifier: BSD-3-Clause
    def get_wind_nccs(self):
        position = self.position
        ###########################################
        # First we construct a TDSCatalog instance pointing to our dataset of interest, in
        # this case TDS' "Best" virtual dataset for the GFS global 0.5 degree collection of
        # GRIB files. We see this catalog contains a single dataset.
        best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/'
                      'Global_0p5deg/catalog.xml?dataset=grib/NCEP/GFS/Global_0p5deg/Best')
        ###########################################
        # We pull out this dataset and get the NCSS access point
        best_ds = best_gfs.datasets[0]
        ncss = best_ds.subset()
        ###########################################
        # We can then use the `ncss` object to create a new query object, which
        # facilitates asking for data from the server.
        query = ncss.query()
        ###########################################
        # We construct a query asking for data corresponding to latitude 40N and longitude 105W,
        # for the next 7 days. We also ask for NetCDF version 4 data, for the variable
        # 'Temperature_isobaric', at the vertical level of 100000 Pa (approximately surface).
        # This request will return all times in the range for a single point. Note the string
        # representation of the query is a properly encoded query string.
        now = datetime.utcnow()
        #position[0] = lon, position[1] = lat
        query.lonlat_point(position[0], position[1]).time(datetime.utcnow())
        ###########################################
        # We now request data from the server using this query. The `NCSS` class handles parsing
        # this NetCDF data (using the `netCDF4` module). If we print out the variable names, we
        # see our requested variables, as well as a few others (more metadata information)
        query.variables('u-component_of_wind_isobaric', 'v-component_of_wind_isobaric')
        query.accept('netcdf')
        data = ncss.get_data(query)
        query = ncss.query()
        u_wind = data.variables['u-component_of_wind_isobaric']
        v_wind = data.variables['v-component_of_wind_isobaric']
        pressure = data.variables['isobaric']
        return u_wind, v_wind, pressure

    def get_wind_data(self, array):
        nparray = np.array(0)
        df = pd.DataFrame()
        for idx in array:
            nparray = idx.flatten()
        return nparray.astype(float)

    def get_wind_df(self):
        parameters = self.get_wind_nccs()
        names = ['u_wind, m/s', 'v_wind, m/s', 'pressure, Pa']
        df = pd.DataFrame()
        for name, parameter in zip(names, parameters):
            df[name] = self.get_wind_data(parameter)
        return df 
    
    def get_wind_full_df(self):
        df = self.get_wind_df()
        directions = []
        speeds = []
        for ix in range(len(df.index)):
            u_wind2 = df['u_wind, m/s'].iloc[ix]*(units.m/ units.s)
            v_wind2 = df['v_wind, m/s'].iloc[ix]*(units.m / units.s)
            direction = mc.wind_direction(u_wind2,v_wind2)
            speed = mc.wind_speed(u_wind2,v_wind2)
            directions.append(direction)
            speeds.append(speed)
        df['direction'] = directions
        df['speed'] = speeds
        return df
    
#     def get_wind_dir_at_pres(self):
#         pressure = self.pressure
#         df = self.get_wind_full_df()
#         direction = df.loc[df['pressure, Pa'] == pressure]
#         return direction
    
    def get_wind_interpolate(self, name):
        value = self.pressure
        df = self.get_wind_full_df()
        #interpolation function given any pressure value return interpolated wind speed or direction
        exactmatch=df[df['pressure, Pa']==value]
        if (exactmatch.empty == False):
            return exactmatch[name].squeeze()
        else:
            df1 = df[df['pressure, Pa']<value]
            df2 = df[df['pressure, Pa']>value]
            df_min = df1[df1['pressure, Pa'] == df1['pressure, Pa'].max()]
            df_max = df2[df2['pressure, Pa'] == df2['pressure, Pa'].min()]
            x_min = df_min['pressure, Pa'].values
            x_max = df_max['pressure, Pa'].values
            y_min = (df_min[name]).values
            y_max = (df_max[name]).values
            dy = y_max - y_min
            dx = x_max - x_min
            A = y_min - (dy/dx) * x_min
            B = dy/dx
            y_int = A + B * value
            return y_int.squeeze()
        
    def get_wind_direction(self):
        return self.get_wind_interpolate('direction')
    
    def get_wind_speed(self):
        return self.get_wind_interpolate('speed')
    

class Baloon(Wind):
    def __init__(self):
        self.id = 0
        self.position = np.array([23,54], dtype=np.float32)
        self.speed = 0
        self.elevation = 0
        self.heading = 0  # rad
        vx = self.speed * math.cos(self.heading)
        vy = self.speed * math.sin(self.heading)
        vz = self.elevation
        self.velocity = np.array([vx, vy, vz], dtype=np.float32)
        self.prev_a = 0   
    
    #just testing
    def get_speed(self):
        wind.position = self.position
        x = wind.get_wind_speed()
        return x
    


Testing

In [ ]:
wind = Wind()
ball = Baloon()
ball.position=[23,20]
wind.pressure = 201
wind.speed = wind.get_wind_speed()
ball.get_speed()